In [1]:
!pip install spacy && python -m spacy download en

✔ Download and installation successful
You can now load the model via spacy.load('en_core_web_sm')
✔ Linking successful
/usr/local/lib/python3.7/dist-packages/en_core_web_sm -->
/usr/local/lib/python3.7/dist-packages/spacy/data/en
You can now load the model via spacy.load('en')


In [2]:
!pip install sentence-transformers

     |████████████████████████████████| 81kB 9.4MB/s 
     |████████████████████████████████| 2.3MB 19.1MB/s 
     |████████████████████████████████| 1.2MB 46.7MB/s 
     |████████████████████████████████| 901kB 42.4MB/s 
     |████████████████████████████████| 3.3MB 47.4MB/s 
  Created wheel for sentence-transformers: filename=sentence_transformers-1.1.1-cp37-none-any.whl size=123338 sha256=b877e35c69a1b2a480b0bf65255d14b264db917d87f8899a7b4e0195fc72598b
  Stored in directory: /root/.cache/pip/wheels/5e/89/29/45e45adc162b50f97f71801e8b07947c9cfe2b3ae7dbf37896
Successfully built sentence-transformers


In [3]:
from pathlib import Path
import os, re, io
import pandas as pd
import numpy as np
import requests

from gensim.parsing.preprocessing import remove_stopwords
from nltk.stem import WordNetLemmatizer
from nltk import word_tokenize
import spacy
nlp = spacy.load('en_core_web_sm')

from sklearn.metrics.pairwise import cosine_similarity

import random
import pickle

from sentence_transformers import SentenceTransformer, models, InputExample, losses
from torch.utils.data import DataLoader

In [4]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [13]:
folder="/content/drive/MyDrive/Sem 3-2 Online/Question Answering System/Final/Code/1-5 DistilBERT"

In [14]:
QA_df = pd.read_csv(folder+'/faqir_faq.csv',header=[0],index_col=0)
QA_df.head()

,FaqID,FaqQuestion,FaqAnswer
0,8656,"How to remove tree sap from car? Hi,I have som...",I have had good luck with commercial products ...
1,40807,How do I get a sticker off of a car window (p...,Goo Gone is a fantastic product (available at ...
2,57727,How to get rid of garbage disposal odor?,1. Inspect the black rubber cover that fits in...
3,120493,How to maintain a lawn mower?,"To properly maintain your lawnmover, you'll ne..."
4,62732,How to fix vertical blinds? I can no longer re...,Click on the link and follow the steps.


In [15]:
class TextPreprocessor():
    def __init__(self, data_df, column_name=None):
        self.data_df = data_df  
        if not column_name and type(colum_name) == str:
            raise Exception("column name is mandatory. Make sure type is string format")
        self.column = column_name
        self.convert_lowercase()    
        self.applied_stopword = False
        self.processed_column_name = f"processed_{self.column}"
        
    def convert_lowercase(self):
        self.data_df.fillna('',inplace=True)
        self.data_df = self.data_df.apply(lambda column: column.astype(str).str.lower(), axis=0)    

    def remove_question_no(self):
        for c in self.column:
            self.data_df[c] = self.data_df[c].apply(lambda row: re.sub(r'^\d+[.]',' ', row))    
        
    def remove_symbols(self):
        for c in self.column:
            self.data_df[c] = self.data_df[c].apply(lambda row: re.sub(r'[^A-Za-z0-9\s]', ' ', row))    

    def process(self):
        self.remove_question_no()
        self.remove_symbols()  
        return self.data_df

In [16]:
text_preprocessor = TextPreprocessor(QA_df.copy(), column_name=["FaqQuestion", "FaqAnswer"])
processed_QA_df = text_preprocessor.process()
processed_QA_df.to_csv(folder+'/processed_QA_df.csv')

In [23]:
qParaphrase_df = pd.read_csv(folder+"/faqir_paraphrases.tsv", sep='\t')
qParaphrase_df.columns=['Query', 'Paraphrase']

In [24]:
text_preprocessor = TextPreprocessor(qParaphrase_df.copy(), column_name=['Query', 'Paraphrase'])
processed_qParaphrase_df = text_preprocessor.process()
processed_qParaphrase_df.to_csv(folder+'/processed_qParaphrase_df.csv')

In [25]:
def qQ_training_dataset_builder():
  qQ_training_dataset=pd.DataFrame(columns=['Query','Paraphrase','Label'],dtype="object")

  temp={'Query':processed_qParaphrase_df['Query'], 'Paraphrase':processed_qParaphrase_df['Paraphrase'], 'Label':[1]*len(processed_qParaphrase_df)}
  temp_df = pd.DataFrame(temp) 
  qQ_training_dataset=temp_df

  paraphrase_dash=[]
  for i in range(len(processed_qParaphrase_df['Query'])):
    paraphrase_dash.append(processed_QA_df.sample()['FaqQuestion'].values[0])
  temp={'Query':processed_qParaphrase_df['Query'], 'Paraphrase':paraphrase_dash, 'Label':[0]*len(processed_qParaphrase_df)}
  temp_df = pd.DataFrame(temp) 
  qQ_training_dataset=qQ_training_dataset.append(temp_df, ignore_index=True)
  return qQ_training_dataset

In [26]:
qQ_training_dataset=qQ_training_dataset_builder()
qQ_training_dataset.to_csv(folder+'/qQ_training_dataset.csv')

In [27]:
train_data = []
for j in qQ_training_dataset.index:
  train_data.append(InputExample(texts=[qQ_training_dataset['Query'][j], qQ_training_dataset['Paraphrase'][j]], label=qQ_training_dataset['Label'][j]))

In [28]:
word_embedding_model = models.Transformer('distilbert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension(),
                               pooling_mode_mean_tokens=True,
                               pooling_mode_cls_token=False,
                               pooling_mode_max_tokens=False)

qQ_model = SentenceTransformer(modules=[word_embedding_model, pooling_model])

Some weights of the model checkpoint at distilbert-base-uncased were not used when initializing DistilBertModel: ['vocab_layer_norm.weight', 'vocab_projector.weight', 'vocab_projector.bias', 'vocab_layer_norm.bias', 'vocab_transform.weight', 'vocab_transform.bias']
- This IS expected if you are initializing DistilBertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DistilBertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [29]:
train_dataloader = DataLoader(train_data, shuffle=True, batch_size=8)
train_loss = losses.SoftmaxLoss(model=qQ_model, sentence_embedding_dimension=qQ_model.get_sentence_embedding_dimension(), num_labels=2)

In [30]:
qQ_model.fit(train_objectives=[(train_dataloader, train_loss)],
          epochs=1,
          evaluation_steps=1000,
          warmup_steps=100)

In [31]:
pickle.dump(qQ_model, open(folder+'/qQ_model.sav', 'wb'))